<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-cleansing" data-toc-modified-id="Data-cleansing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data cleansing</a></span></li><li><span><a href="#Timestamp" data-toc-modified-id="Timestamp-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Timestamp</a></span></li><li><span><a href="#Rating" data-toc-modified-id="Rating-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Rating</a></span></li><li><span><a href="#Data-Types" data-toc-modified-id="Data-Types-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data Types</a></span></li></ul></div>

In [246]:
import re
import pandas as pd

In [247]:
rdb = pd.read_excel('redditwhiskeydatabase.xlsx')

In [248]:
rdb = rdb.reset_index().rename({'index':'reviewID',
                          'Timestamp':'timestamp',
                          'Whisky Name': 'whisky',
                          'Reviewer\'s Reddit Username':'username',
                          'Link To Reddit Review': 'reviewLink',
                          'Reviewer Rating': 'rating',
                          'Whisky Region or Style':'style',
                          'Full Bottle Price Paid':'pricePaid',
                          'Date of Review':'date'
                         },axis='columns').set_index('reviewID')

Add ID and rename columns

## Data cleansing

## Timestamp

Some timestamps aren't actually timestamps:

In [249]:
rdb[rdb.timestamp.str.contains('[a-zA-Z]',na=False)]

,timestamp,whisky,username,reviewLink,rating,style,pricePaid,date
reviewID,,,,,,,,
7885,Bunnahabhain 25,Bunnahabhain 25,reddbdb,http://www.reddit.com/r/Scotch/comments/2t1y1h...,83,Islay,350,01/19/15
30096,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review


Fix date

In [250]:
rdb.loc[rdb.timestamp == 'Bunnahabhain 25',['timestamp']] = '01/19/15'

And delete the other one, that is just the headings:

In [251]:
rdb = rdb[rdb['whisky'] != 'Whisky Name']

Check to see:

In [252]:
rdb[rdb.timestamp.str.contains('[a-zA-Z]',na=False)]

,timestamp,whisky,username,reviewLink,rating,style,pricePaid,date
reviewID,,,,,,,,


## Rating

In [253]:
rdb[rdb.rating.str.contains('[a-zA-Z]',na=False)]

,timestamp,whisky,username,reviewLink,rating,style,pricePaid,date
reviewID,,,,,,,,
10264,2018-11-11 07:09:31.985000,CT Branford Super Bourbon Whiskey,Bourbonexpert,https://www.reddit.com/r/bourbon/comments/734x...,100/100. This the best Bourbon I have ever tas...,Bourbon/America,20.99,11/11/2018
20179,2019-02-15 15:51:29.291000,Kilchoman New Make Spirit,j4ni,https://old.reddit.com/r/Scotch/comments/74mj4...,na,Islay,NaN,10/06/17


Remove the /100 from some reviews:

In [254]:
rdb['rating'] = rdb['rating'].replace('\/100(.)*','',regex=True)

Remove 'na's and blanks, as well as values where the is a date instead of a rating:

In [255]:
rdb = rdb[rdb.rating != 'na']
rdb = rdb.dropna(subset=['rating'])
rdb = rdb[rdb.rating != ' ']
rdb = rdb[rdb.rating != '07/25/18']

Delete value where there's a date instead of a rating:

In [256]:
rdb.rating.unique()

array([68, 8, 80, 90, 85, 69, 78, 87, 84, 75, 82, 83, 89, 86, 79, 76, 88,
       64, 74, 71, 77, 7, 60, 70, 65, 81, 72, 55, 95, 58, 93, 50, 10, 73,
       91, 66, 96, 92, 94, 97, '90', 63, 67, 41, 45, 54, 62, 86.5, 87.5,
       61, 88.5, 59, 100, 35, 30, 3.5, 98, 52, 40, 99, 53, 83.5, 57, 36,
       14, 48, 47, 51, 20, 85.5, 37, 56, 0.1, 17, 42, 82.5, 23, 43, 84.5,
       95.5, 91.5, 94.5, 19, 90.5, 49, 25, 15, 11, '75', 39, '100', 24,
       32, 44, 1, '88', 31, 38, 33, 18, 46, 29, 6, 22, 34, 28, 2, '93', 5,
       '70', 3, 9, 89.5, 6.5, '91'], dtype=object)

## Data Types

Correct the datatypes for all fields.

In [260]:
rdb.dtypes

timestamp     object
whisky        object
username      object
reviewLink    object
rating        object
style         object
pricePaid     object
date          object
dtype: object

In [270]:
rdb['timestamp'] = pd.to_datetime(rdb.timestamp)
rdb['whisky'] = rdb['whisky'].astype(str)
rdb['username'] = rdb['username'].astype(str)
rdb['reviewLink'] = rdb['reviewLink'].astype(str)
rdb['rating'] = rdb['rating'].astype(int)
rdb['style'] = rdb['style'].astype(str)
rdb['pricePaid'] = rdb['pricePaid'].astype(str)
rdb['date'] = pd.to_datetime(rdb.date,errors='coerce')

# Add submission IDs

To use the reddit api we need the submission ids from the links.

In [274]:
def submission_id(link):
    # get the submission id from the link to the reddit thread
    try:
        id_text = re.search('comments/.*?/',link).group()
        #split on '/', and get the second value which is the subid'
        sub_id = id_text.split('/')[1]
    except:
        sub_id = ''
    return sub_id

In [273]:
rdb['sub_id'] = rdb['reviewLink'].apply(submission_id)

comments/14uder/
comments/9ssqyk/
comments/7445vx/
comments/9ryi8f/
comments/a7jwdd/
comments/5aezxj/
comments/6epezn/
comments/6ijts3/
comments/6kaxx1/
comments/56f1km/
comments/6p48zm/
comments/5hlxwt/
comments/52b8vd/
comments/54s1ff/
comments/74hf7g/
comments/79w2fs/
comments/7ajpxm/
comments/7768ib/
comments/53u25y/
comments/8gypt5/
comments/5ufyw2/
comments/9ip8oo/
comments/7ru5sz/
comments/863eqq/
comments/6sm7k0/
comments/7ajpxm/
comments/7c4kfc/
comments/a7jwdd/
comments/67a74d/
comments/7am8g5/
comments/6jw7mm/
comments/5wli4x/
comments/788fyy/
comments/6fbtzu/
comments/3v2y9m/
comments/410u1q/
comments/72el8n/
comments/3rywqp/
comments/4a6xf8/
comments/4ros37/
comments/57n6gk/
comments/7768ib/
comments/7d83qg/
comments/4l5ako/
comments/6tw4ee/
comments/8ciynv/
comments/9v9ej4/
comments/2i3for/
comments/2w5llu/
comments/2fy7v2/
comments/1l1070/
comments/4pbgpt/
comments/33610q/
comments/2flmuj/
comments/202yyw/
comments/3s7fir/
comments/3ern3l/
comments/1tl4ih/
comments/z47f7

comments/14fkoa/
comments/1ig30b/
comments/2o1n65/
comments/1zo5t3/
comments/504itc/
comments/2ge839/
comments/1ft9k8/
comments/2o1n65/
comments/222udo/
comments/30hpwt/
comments/1njiqq/
comments/3pxp1i/
comments/55zhdg/
comments/16dtus/
comments/20y6tz/
comments/1o6g8f/
comments/uar37/
comments/23icgd/
comments/pob86/
comments/2cstqg/
comments/2o1n65/
comments/1pueo5/
comments/40nmkv/
comments/mcu2z/
comments/riurf/
comments/2xaobh/
comments/1iscqk/
comments/28f4ye/
comments/1uri6o/
comments/z19xg/
comments/31wbic/
comments/73jwhh/
comments/15y7jg/
comments/26u1zu/
comments/2gkpe4/
comments/wla3h/
comments/1xgkq4/
comments/3ibkr2/
comments/13z38m/
comments/66gz8a/
comments/2o1n65/
comments/1ajkgf/
comments/2cdtmb/
comments/1ilpbi/
comments/5r7yqs/
comments/44mvz1/
comments/7acyul/
comments/40vz0d/
comments/7c075v/
comments/7gss53/
comments/7xp86j/
comments/81ilnl/
comments/83fvaq/
comments/8jx0iu/
comments/8n0py2/
comments/9cemq0/
comments/9gpb7l/
comments/9lqe3e/
comments/98mmyc/
com

comments/wzj4n/
comments/7cek1x/
comments/8s3zl8/
comments/8xrg5e/
comments/93834r/
comments/8jiywd/
comments/5lkfu5/
comments/6me56u/
comments/5oq2vs/
comments/2gwdnc/
comments/291bji/
comments/5vnrs2/
comments/1s9v07/
comments/326osu/
comments/3zgclw/
comments/1wvznd/
comments/2k46gq/
comments/6kfilw/
comments/6me56u/
comments/a6qq5z/
comments/4yjzcn/
comments/2so2uc/
comments/3zb62c/
comments/6me56u/
comments/8b3iam/
comments/508qav/
comments/2px4ll/
comments/2ox32k/
comments/2rcnp4/
comments/3qlg85/
comments/6me56u/
comments/1vmb3a/
comments/80r22n/
comments/35bvo4/
comments/66tj74/
comments/4su0st/
comments/50i5d8/
comments/4ydxy0/
comments/77e202/
comments/7g2xm7/
comments/93834r/
comments/9i8f5s/
comments/ae4b06/
comments/56qtdd/
comments/6nazcd/
comments/6me56u/
comments/6nt0qm/
comments/6nazcd/
comments/6me56u/
comments/6rk519/
comments/4l84fd/
comments/53wmwc/
comments/5w2yte/
comments/6me56u/
comments/5wvhvc/
comments/8x6y7x/
comments/896azv/
comments/8pplm0/
comments/bc60tt

comments/1hvgaw/
comments/44iogj/
comments/1f5yds/
comments/4yik71/
comments/2v0nzp/
comments/1i2fd6/
comments/31qxpd/
comments/2y7kq9/
comments/1roqtz/
comments/21yr7f/
comments/1dgq4o/
comments/2rswkk/
comments/1uxc0p/
comments/2zlqe9/
comments/1iyg9z/
comments/5661tf/
comments/46xdpm/
comments/5nopz3/
comments/5it3ba/
comments/5i2ytn/
comments/1ffzkt/
comments/87x2er/
comments/3fjb11/
comments/3jmyrt/
comments/5xq87r/
comments/3qu1tn/
comments/6z3f7e/
comments/anb1py/
comments/1uwfi5/
comments/19dmno/
comments/q67mv/
comments/16ou9h/
comments/2apgh0/
comments/21ptub/
comments/200fsv/
comments/y19sy/
comments/2g7v1a/
comments/26tash/
comments/id11x/
comments/26p8x2/
comments/2jar6r/
comments/59ioer/
comments/12rff3/
comments/1suga6/
comments/4bu9j9/
comments/1f5yds/
comments/1uylbc/
comments/55tv9t/
comments/x4z3k/
comments/2xd51o/
comments/1uidvx/
comments/x82os/
comments/oditp/
comments/vberq/
comments/4gkyw5/
comments/4k0vzt/
comments/18cqu9/
comments/1hv0dd/
comments/15ixlp/
comm

comments/1gtzp4/
comments/7iof5f/
comments/3xoizv/
comments/4oon0o/
comments/4qer6l/
comments/4jz3mf/
comments/3xoizv/
comments/pgb65/
comments/23mcsh/
comments/54q2k6/
comments/192iwv/
comments/4qer6l/
comments/7q7hil/
comments/bf4u3u/
comments/3dsz76/
comments/3eoc2k/
comments/az57w8/
comments/5fgotr/
comments/7q7hil/
comments/98on1s/
comments/3zgg71/
comments/3eoc2k/
comments/2v5y7o/
comments/okgnq/
comments/192iwv/
comments/3fxb90/
comments/741yyt/
comments/2rca6o/
comments/4i536k/
comments/7iof5f/
comments/2jcau5/
comments/4ffrdj/
comments/3a8gwc/
comments/523m25/
comments/3mylv7/
comments/7xurvk/
comments/7amgsf/
comments/2gw6de/
comments/t7bty/
comments/2twcr3/
comments/3eoc2k/
comments/3q2nq4/
comments/1oi459/
comments/3nu1q2/
comments/5bx040/
comments/7iof5f/
comments/30z8r2/
comments/2g46yk/
comments/33y7s9/
comments/20s0d3/
comments/9e557n/
comments/9sgw1v/
comments/83npi5/
comments/53qnje/
comments/9sk3uk/
comments/3vsj9w/
comments/6vjil3/
comments/2jba3w/
comments/1sblcp/


...

comments/1ju7p5/
comments/2xqbfk/
comments/20718l/
comments/1twxu5/
comments/owfxv/
comments/59lfd7/
comments/2wkhni/
comments/7pcsft/
comments/83hy70/
comments/86f6iw/
comments/8j9zkb/
comments/9tmr0o/
comments/au8g89/
comments/7ifl3d/
comments/33fvzz/
comments/8jpi1n/
comments/85onc6/
comments/828y1p/
comments/15gc6w/
comments/221dan/
comments/18h3wf/
comments/1h7f2g/
comments/181qfs/
comments/1a2eh0/
comments/1jig7l/
comments/1m3n2u/
comments/7tutmx/
comments/7nkkrn/
comments/7tv59r/
comments/68qwn6/
comments/4kan5g/
comments/8g3ixt/
comments/a38ed2/
comments/5jeka1/
comments/bdbdkw/
comments/3uidu9/
comments/2ykatl/
comments/4pbxu9/
comments/41ew9c/
comments/4bymfi/
comments/4kny2p/
comments/8kf97q/
comments/9v4jwu/
comments/al649q/
comments/6t24j7/
comments/9aujdr/
comments/8ec1g7/
comments/ah0wgl/
comments/apuwcl/
comments/afwh8l/
comments/8l0chw/
comments/8795kq/
comments/adbdz3/
comments/79i8t0/
comments/7un4r6/
comments/90qke8/
comments/4wx8z3/
comments/61wvly/
comments/2ptd4w

comments/20iqtx/
comments/109oa5/
comments/3t56kk/
comments/3t56kk/
comments/2vsjpo/
comments/2xtec4/
comments/2cz9ze/
comments/6894fu/
comments/1ztam9/
comments/7t50xn/
comments/7xnenj/
comments/87ivrt/
comments/9ugjyz/
comments/a86bn6/
comments/aki2cf/
comments/6ypcnw/
comments/2xaaxi/
comments/601wsn/
comments/5vzga5/
comments/5bj07x/
comments/2xyayu/
comments/2h7t4r/
comments/73qzq1/
comments/4tudtx/
comments/5zx1go/
comments/7849af/
comments/78p6xn/
comments/49cmgc/
comments/7lsmrg/
comments/80esin/
comments/8jigrc/
comments/7ovpjc/
comments/9eh6d4/
comments/2h7t4r/
comments/2624d4/
comments/146iah/
comments/2h7t4r/
comments/16anhi/
comments/utpp7/
comments/2jcau5/
comments/3ro7ev/
comments/18qlmn/
comments/2l8675/
comments/6p5vzn/
comments/854vm1/
comments/aooewk/
comments/1do9uz/
comments/18ji4n/
comments/2624d4/
comments/407n0s/
comments/pbegc/
comments/4l0ol6/
comments/1fdfy9/
comments/2y2cht/
comments/63ky4t/
comments/1bwy7m/
comments/1ztam9/
comments/7lsmrg/
comments/7quf3v/

comments/mr7dm/
comments/30qozg/
comments/qzdy3/
comments/4yfxtw/
comments/2u3pgh/
comments/2o1myl/
comments/22o7sj/
comments/1z25ul/
comments/4gtlan/
comments/q69xr/
comments/1xgf0d/
comments/6zf13h/
comments/wm1pg/
comments/78suk8/
comments/8700ek/
comments/8akx46/
comments/9yk8de/
comments/aa08j4/
comments/1mjilf/
comments/302edf/
comments/3vehrn/
comments/3exdiu/
comments/8wyagl/
comments/anqehu/
comments/83cbel/
comments/8bvydo/
comments/7yzvng/
comments/3xu6im/
comments/1sbyfo/
comments/97c34t/
comments/32qwda/
comments/70lmuq/
comments/7i0kzm/
comments/4p4k3r/
comments/5qeahy/
comments/4d1505/
comments/5hb4nd/
comments/4v0mmy/
comments/63s34e/
comments/4vy6mm/
comments/4pdmwf/
comments/4lz3fd/
comments/90agff/
comments/4y5edm/
comments/57g712/
comments/8ukx8a/
comments/53njw8/
comments/9b4wmd/
comments/4f9qhp/
comments/4k2brx/
comments/a15gne/
comments/aj0f57/
comments/axyc8e/
comments/71wiox/
comments/8s3sox/
comments/3ct9o0/
comments/3g2b14/
comments/58vaoc/
comments/5em945/
c

comments/6ohape/
comments/11g0g7/
comments/3prg7b/
comments/6vcc7b/
comments/83p96c/
comments/4bzufn/
comments/6v3go9/
comments/6v6dgm/
comments/737m12/
comments/6v3vn3/
comments/6iioof/
comments/5v10jf/
comments/4zqt9h/
comments/5mx8lo/
comments/7ehkr4/
comments/bakxh8/
comments/ajx851/
comments/6vyzof/
comments/6h3kau/
comments/9l5g79/
comments/3kh403/
comments/as5tjg/
comments/6zhnpi/
comments/8lotdq/
comments/578ak2/
comments/5gy559/
comments/8iixgc/
comments/8k3zy0/
comments/aeul6o/
comments/938iit/
comments/7t74rf/
comments/93kcvf/
comments/7x078u/
comments/ajx851/
comments/5e750u/
comments/8lswia/
comments/6lttil/
comments/5n5b2f/
comments/8966pu/
comments/7cnqn1/
comments/be6bn3/
comments/7i6yn1/
comments/a2kxew/
comments/5tonnn/
comments/86oc2o/
comments/7841a0/
comments/86oc2o/
comments/6ezxv9/
comments/9rbyzr/
comments/antpul/
comments/8diezt/
comments/93e7jn/
comments/80m61l/
comments/5zybqh/
comments/6llh21/
comments/6ynaub/
comments/776dme/
comments/8t107i/
comments/adnf0

comments/30v03g/
comments/6cvu3f/
comments/3yy9u9/
comments/6d46c0/
comments/6birjd/
comments/88jtha/
comments/6cpv72/
comments/6c0qz7/
comments/46cbdk/
comments/8wkeu5/
comments/90frio/
comments/92j28c/
comments/94k9p5/
comments/978745/
comments/3qcks9/
comments/5zfe29/
comments/4gs7fz/
comments/3h8nof/
comments/68p0e6/
comments/6pd5bj/
comments/7snpk5/
comments/8r4673/
comments/8foiol/
comments/6rhkon/
comments/68p0e6/
comments/6pd5bj/
comments/7ywe04/
comments/aryj3o/
comments/b9jm59/
comments/913k86/
comments/94k9p5/
comments/99ehor/
comments/8pgqsz/
comments/1i3id7/
comments/2j2e48/
comments/4ocb9g/
comments/1v324s/
comments/1zt6ox/
comments/2ts5od/
comments/4rscqb/
comments/5lp0vz/
comments/28rd3s/
comments/28qull/
comments/1elmwl/
comments/2nzyrb/
comments/21ae5y/
comments/1kr8x4/
comments/3agjni/
comments/4l2dn4/
comments/24qt5n/
comments/2tenn0/
comments/2w8dkc/
comments/2w8dkc/
comments/4jeem4/
comments/6zopr5/
comments/49r6hf/
comments/2i1r4r/
comments/4anra4/
comments/2pbbl

In [275]:
rdb.head()

,timestamp,whisky,username,reviewLink,rating,style,pricePaid,date,sub_id
reviewID,,,,,,,,,
0,2012-12-14 10:03:18.000,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,nan,2012-12-14,14uder
1,2018-11-04 00:50:38.789,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,nan,2018-10-30,9ssqyk
2,2017-11-17 08:15:21.922,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,nan,2017-10-17,7445vx
3,2018-11-04 00:49:14.921,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,34.99,2018-10-27,9ryi8f
4,2018-12-19 00:38:02.415,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,nan,2018-12-19,a7jwdd


Save to parquet

In [276]:
rdb.to_parquet('redditwhiskeydatabase.parquet')